# Homework : 7 Machine Learning - 1 (Supervised Methods)

## c) For the best hyper-parameters found in part (b), run 5 training runs out to 100 epochs. Report the best accuracy (over epochs) on val for each run - this is 5 numbers. Compute, mean, max, and std deviation for these 5 values.

### Importing all necessary libraries

In [1]:
import time
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras import Model

from utils.engine import Classifier, MyModel

tf.config.run_functions_eagerly(True)

print("TensorFlow version:", tf.__version__)

2023-04-07 15:52:15.315882: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.12.0


### Loading the saved training and test data


In [2]:
train_images = np.load("./dataset/Fashion_Mnist/train_images.npy")
train_labels = np.load("./dataset/Fashion_Mnist/train_labels.npy")
test_images = np.load("./dataset/Fashion_Mnist/test_images.npy")
test_labels = np.load("./dataset/Fashion_Mnist/test_labels.npy")

### Instantiating an object for the Classifier class in engine.py

In [3]:
clf = Classifier()

### Splitting the training data into train data and validation data (80/20 split)

In [4]:
X_test = test_images
T_test = test_labels
(X_train, T_train), (X_validation, T_validation) = clf.splitTrainData(train_images, train_labels, test_images, validation_split=0.2, printFlag=True)

---------------------------------------------------------------------------------------------
Total number of images in Training data:  48000
Total number of images in Validation data:  12000
Total number of images in Test data:  10000
Total number of classes in the output lables: 10
---------------------------------------------------------------------------------------------


### Normalizing the pixel values.

In [5]:
X_train, X_validation, X_test = X_train / 255.0, X_validation / 255.0, X_test / 255.0


## Getting the best hyper-parameters from the "finalCombo.pkl" file saved in the "Results" Directory and also fixing the batch size for the training and validation data.

In [15]:
# read python dict back from the file
pkl_file = open('./Result/finalCombo.pkl', 'rb')
final_combination = pickle.load(pkl_file)
pkl_file.close()

learning_rate = final_combination["learning_rate"]
hidden_units = final_combination["hidden_units"]
lambd_a = final_combination["reg_param"]
print(f"Hidden Units: {hidden_units}, Learning Rate: {learning_rate}, Regularization Parameter: {lambd_a}")

Hidden Units: 160, Learning Rate: 0.1, Regularization Parameter: 0.001


In [16]:
train_ds, validation_ds = clf.setBatchSize(32, X_train, T_train, X_validation, T_validation)

### Training the model with best hyper-parameters for 5 iterations with 100 epochs per iteration and tracking the best validation accuracy over the 100 epochs for every iteration.

In [17]:
my_model = MyModel(hidden_units, lambd_a, 784)
my_model.build((None, 28, 28, 1)) 
my_model.summary()

Model: "my_model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Layer (Flatten)       (None, 784)               0         
                                                                 
 Hidden_Layer (Dense)        (None, 160)               125600    
                                                                 
 Output_Layer (Dense)        (None, 10)                1610      
                                                                 
Total params: 127,210
Trainable params: 127,210
Non-trainable params: 0
_________________________________________________________________


In [9]:
best_acc_list = []
epochs_list = []
for i in range(5):
    print("########################################################################################################################################################################################################")
    print(f"Iteration ======> {i+1}")
    result = clf.training_loop_c(train_ds, validation_ds, hidden_units, lambd_a, learning_rate, 100, i+1)
    print(f"Highest Validation Accuracy: {result['best_validation_accuracy']}% at Epoch: #{result['epoch']}")
    best_acc_list.append(result["best_validation_accuracy"])
    epochs_list.append(result["epoch"])
    print()

########################################################################################################################################################################################################
Iteration ======> 1
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


2023-04-07 15:52:26.691822: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [48000]
	 [[{{node Placeholder/_1}}]]
2023-04-07 15:52:50.266867: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [12000]
	 [[{{node Placeholder/_1}}]]


Epoch 1, Training Loss: 0.5535715222358704, Training Accuracy: 80.32500457763672, Validation Loss: 0.4823920428752899, Validation Accuracy: 82.78333282470703
Model saved for EPOCH: 1 at ITERATION: 1 with Validation Accuracy: 82.78333282470703%
Highest Accuracy so far: 82.78333282470703% at EPOCH: 1 in ITERATION: 1

Epoch 2, Training Loss: 0.40794727206230164, Training Accuracy: 85.16458129882812, Validation Loss: 0.4017268121242523, Validation Accuracy: 85.4749984741211
Model saved for EPOCH: 2 at ITERATION: 1 with Validation Accuracy: 85.4749984741211%
Highest Accuracy so far: 85.4749984741211% at EPOCH: 2 in ITERATION: 1

Epoch 3, Training Loss: 0.36746424436569214, Training Accuracy: 86.64583587646484, Validation Loss: 0.3701515197753906, Validation Accuracy: 86.69166564941406
Model saved for EPOCH: 3 at ITERATION: 1 with Validation Accuracy: 86.69166564941406%
Highest Accuracy so far: 86.69166564941406% at EPOCH: 3 in ITERATION: 1

Epoch 4, Training Loss: 0.341843843460083, Trainin

In [12]:
best_acc_list


[<tf.Tensor: shape=(), dtype=float32, numpy=88.9>,
 <tf.Tensor: shape=(), dtype=float32, numpy=88.74167>,
 <tf.Tensor: shape=(), dtype=float32, numpy=88.566666>,
 <tf.Tensor: shape=(), dtype=float32, numpy=88.941666>,
 <tf.Tensor: shape=(), dtype=float32, numpy=88.625>]

In [13]:
epoch_maxAcc_map = dict(zip(epochs_list, best_acc_list))
epoch_maxAcc_map

{13: <tf.Tensor: shape=(), dtype=float32, numpy=88.9>,
 35: <tf.Tensor: shape=(), dtype=float32, numpy=88.74167>,
 16: <tf.Tensor: shape=(), dtype=float32, numpy=88.566666>,
 19: <tf.Tensor: shape=(), dtype=float32, numpy=88.941666>,
 20: <tf.Tensor: shape=(), dtype=float32, numpy=88.625>}

### Computing Mean, Max and Standard Deviation for the 5 Highest Validation Accuracies computed.

In [14]:
mean_val_acc = np.mean(best_acc_list)
std_val_acc = np.std(best_acc_list)
max_val_acc = np.max(best_acc_list)
print(f"Mean for the 5 Best Validation Accuracy Values: {mean_val_acc}")
print(f"Standard Deviation for the 5 Best Validation Accuracy Values: {std_val_acc}")
print(f"Maximum Value from the 5 Best Accuracy Values: {max_val_acc}")

Mean for the 5 Best Validation Accuracy Values: 88.75498962402344
Standard Deviation for the 5 Best Validation Accuracy Values: 0.1472528874874115
Maximum Value from the 5 Best Accuracy Values: 88.94166564941406
